In [1]:
import pandas as pd
import requests
import numpy as np
from urllib.parse import urlparse
import time
import os
import openai

This notebook retrieves the legistar client names for cities and towns in California. We get up to 100 free google search API calls per day, and after that it's \\$5 for a thousand calls - so about \\$2.50 to run through the whole list of cities + towns. Won't break us to run it ocassionally, but we shouldn't execute this script mindlessly.

If you need to run this script, you have to set up environment variables. I included them in the Vercel list of environment variables - even though they're not required for deployment of our website, it's still nice to have them in a shared place.

In [2]:
cities = [ # this list is generated by copy/pasting the wikipedia table here: 
    # https://en.wikipedia.org/wiki/List_of_municipalities_in_California
    # and asking chatgpt for a python literal list
    "Adelanto", "Agoura Hills", "Alameda", "Albany", "Alhambra", "Aliso Viejo", "Alturas", "Amador City",
    "American Canyon", "Anaheim", "Anderson", "Angels Camp", "Antioch", "Apple Valley", "Arcadia", "Arcata",
    "Arroyo Grande", "Artesia", "Arvin", "Atascadero", "Atherton", "Atwater", "Auburn", "Avalon", "Avenal",
    "Azusa", "Bakersfield", "Baldwin Park", "Banning", "Barstow", "Beaumont", "Bell", "Bell Gardens",
    "Bellflower", "Belmont", "Belvedere", "Benicia", "Berkeley", "Beverly Hills", "Big Bear Lake", "Biggs",
    "Bishop", "Blue Lake", "Blythe", "Bradbury", "Brawley", "Brea", "Brentwood", "Brisbane", "Buellton",
    "Buena Park", "Burbank", "Burlingame", "Calabasas", "Calexico", "California City", "Calimesa", "Calipatria",
    "Calistoga", "Camarillo", "Campbell", "Canyon Lake", "Capitola", "Carlsbad", "Carmel-by-the-Sea",
    "Carpinteria", "Carson", "Cathedral City", "Ceres", "Cerritos", "Chico", "Chino", "Chino Hills", "Chowchilla",
    "Chula Vista", "Citrus Heights", "Claremont", "Clayton", "Clearlake", "Cloverdale", "Clovis", "Coachella",
    "Coalinga", "Colfax", "Colma", "Colton", "Colusa", "Commerce", "Compton", "Concord", "Corcoran", "Corning",
    "Corona", "Coronado", "Corte Madera", "Costa Mesa", "Cotati", "Covina", "Crescent City", "Cudahy",
    "Culver City", "Cupertino", "Cypress", "Daly City", "Dana Point", "Danville", "Davis", "Del Mar",
    "Del Rey Oaks", "Delano", "Desert Hot Springs", "Diamond Bar", "Dinuba", "Dixon", "Dorris", "Dos Palos",
    "Downey", "Duarte", "Dublin", "Dunsmuir", "East Palo Alto", "Eastvale", "El Cajon", "El Centro", "El Cerrito",
    "El Monte", "El Segundo", "Elk Grove", "Emeryville", "Encinitas", "Escalon", "Escondido", "Etna", "Eureka",
    "Exeter", "Fairfax", "Fairfield", "Farmersville", "Ferndale", "Fillmore", "Firebaugh", "Folsom", "Fontana",
    "Fort Bragg", "Fort Jones", "Fortuna", "Foster City", "Fountain Valley", "Fowler", "Fremont", "Fresno",
    "Fullerton", "Galt", "Garden Grove", "Gardena", "Gilroy", "Glendale", "Glendora", "Goleta", "Gonzales",
    "Grand Terrace", "Grass Valley", "Greenfield", "Gridley", "Grover Beach", "Guadalupe", "Gustine",
    "Half Moon Bay", "Hanford", "Hawaiian Gardens", "Hawthorne", "Hayward", "Healdsburg", "Hemet", "Hercules",
    "Hermosa Beach", "Hesperia", "Hidden Hills", "Highland", "Hillsborough", "Hollister", "Holtville",
    "Hughson", "Huntington Beach", "Huntington Park", "Huron", "Imperial", "Imperial Beach", "Indian Wells",
    "Indio", "City of Industry", "Inglewood", "Ione", "Irvine", "Irwindale", "Isleton", "Jackson", "Jurupa Valley",
    "Kerman", "King City", "Kingsburg", "La Cañada Flintridge", "La Habra", "La Habra Heights", "La Mesa",
    "La Mirada", "La Palma", "La Puente", "La Quinta", "La Verne", "Lafayette", "Laguna Beach", "Laguna Hills",
    "Laguna Niguel", "Laguna Woods", "Lake Elsinore", "Lake Forest", "Lakeport", "Lakewood", "Lancaster",
    "Larkspur", "Lathrop", "Lawndale", "Lemon Grove", "Lemoore", "Lincoln", "Lindsay", "Live Oak", "Livermore",
    "Livingston", "Lodi", "Loma Linda", "Lomita", "Lompoc", "Long Beach", "Loomis", "Los Alamitos", "Los Altos",
    "Los Altos Hills", "Los Angeles", "Los Banos", "Los Gatos", "Loyalton", "Lynwood", "Madera", "Malibu",
    "Mammoth Lakes", "Manhattan Beach", "Manteca", "Maricopa", "Marina", "Martinez", "Marysville", "Maywood",
    "McFarland", "Mendota", "Menifee", "Menlo Park", "Merced", "Mill Valley", "Millbrae", "Milpitas",
    "Mission Viejo", "Modesto", "Monrovia", "Montague", "Montclair", "Monte Sereno", "Montebello", "Monterey",
    "Monterey Park", "Moorpark", "Moraga", "Moreno Valley", "Morgan Hill", "Morro Bay", "Mount Shasta",
    "Mountain House", "Mountain View", "Murrieta", "Napa", "National City", "Needles", "Nevada City", "Newark",
    "Newman", "Newport Beach", "Norco", "Norwalk", "Novato", "Oakdale", "Oakland", "Oakley", "Oceanside",
    "Ojai", "Ontario", "Orange", "Orange Cove", "Orinda", "Orland", "Oroville", "Oxnard", "Pacific Grove",
    "Pacifica", "Palm Desert", "Palm Springs", "Palmdale", "Palo Alto", "Palos Verdes Estates", "Paradise",
    "Paramount", "Parlier", "Pasadena", "Paso Robles", "Patterson", "Perris", "Petaluma", "Pico Rivera",
    "Piedmont", "Pinole", "Pismo Beach", "Pittsburg", "Placentia", "Placerville", "Pleasant Hill", "Pleasanton",
    "Plymouth", "Point Arena", "Pomona", "Port Hueneme", "Porterville", "Portola", "Portola Valley", "Poway",
    "Rancho Cordova", "Rancho Cucamonga", "Rancho Mirage", "Rancho Palos Verdes", "Rancho Santa Margarita",
    "Red Bluff", "Redding", "Redlands", "Redondo Beach", "Redwood City", "Reedley", "Rialto", "Richmond",
    "Ridgecrest", "Rio Dell", "Rio Vista", "Ripon", "Riverbank", "Riverside", "Rocklin", "Rohnert Park",
    "Rolling Hills", "Rolling Hills Estates", "Rosemead", "Roseville", "Ross", "Sacramento", "St. Helena",
    "Salinas", "San Anselmo", "San Bernardino", "San Bruno", "San Carlos", "San Clemente", "San Diego",
    "San Dimas", "San Fernando", "San Francisco", "San Gabriel", "San Jacinto", "San Joaquin", "San Jose",
    "San Juan Bautista", "San Juan Capistrano", "San Leandro", "San Luis Obispo", "San Marcos", "San Marino",
    "San Mateo", "San Pablo", "San Rafael", "San Ramon", "Sand City", "Sanger", "Santa Ana", "Santa Barbara",
    "Santa Clara", "Santa Clarita", "Santa Cruz", "Santa Fe Springs", "Santa Maria", "Santa Monica",
    "Santa Paula", "Santa Rosa", "Santee", "Saratoga", "Sausalito", "Scotts Valley", "Seal Beach", "Seaside",
    "Sebastopol",
    "Selma",
    "Shafter",
    "Shasta Lake",
    "Sierra Madre",
    "Signal Hill",
    "Simi Valley",
    "Solana Beach",
    "Soledad",
    "Solvang",
    "Sonoma",
    "Sonora",
    "South El Monte",
    "South Gate",
    "South Lake Tahoe",
    "South Pasadena",
    "South San Francisco",
    "Stanton",
    "Stockton",
    "Suisun City",
    "Sunnyvale",
    "Susanville",
    "Sutter Creek",
    "Taft",
    "Tehachapi",
    "Tehama",
    "Temecula",
    "Temple City",
    "Thousand Oaks",
    "Tiburon",
    "Torrance",
    "Tracy",
    "Trinidad",
    "Truckee",
    "Tulare",
    "Tulelake",
    "Turlock",
    "Tustin",
    "Twentynine Palms",
    "Ukiah`",
    "Union City",
    "Upland",
    "Vacaville",
    "Vallejo",
    "Ventura",
    "Vernon",
    "Victorville",
    "Villa Park",
    "Visalia",
    "Vista",
    "Walnut",
    "Walnut Creek",
    "Wasco",
    "Waterford",
    "Watsonville",
    "Weed",
    "West Covina",
    "West Hollywood",
    "West Sacramento",
    "Westlake Village",
    "Westminster",
    "Westmorland",
    "Wheatland",
    "Whittier",
    "Wildomar",
    "Williams",
    "Willits",
    "Willows",
    "Windsor",
    "Winters",
    "Woodlake",
    "Woodland",
    "Woodside",
    "Yorba Linda",
    "Yountville",
    "Yreka",
    "Yuba City",
    "Yucaipa",
    "Yucca Valley"
]


In [3]:
import requests
import time
import numpy as np
import pandas as pd
from urllib.parse import urlparse

def find_legistar_urls(cities, google_api_key, cx, openai_token):
    """
    For each city in the provided list, uses Google Custom Search API to search for 
    'legistar {city} california' and extracts the first URL containing 'legistar.com' 
    from the top 5 search results. Then, using OpenAI's API, it checks if the extracted 
    URL seems to belong to the city in question. If not, the URL is set as NaN.
    
    Args:
        cities (list): List of California city names.
        google_api_key (str): Google Custom Search API key.
        cx (str): Google Custom Search engine ID.
        openai_token (str): OpenAI API token.
        
    Returns:
        pd.DataFrame: DataFrame with cities and their corresponding Legistar URLs
                      (NaN if no matching URL found or if deemed unrelated).
    """
    # Set the OpenAI API key
    openai.api_key = openai_token

    results = []
    
    for city in cities:
        query = f"legistar {city} california"
        legistar_url = np.nan
        
        try:
            # Construct the API request URL for Google Custom Search
            search_url = "https://www.googleapis.com/customsearch/v1"
            params = {
                'q': query,
                'key': google_api_key,
                'cx': cx,
                'num': 5  # Request 5 results
            }
            
            # Send the request to Google
            response = requests.get(search_url, params=params)
            
            if response.status_code == 200:
                search_data = response.json()
                if 'items' in search_data:
                    # Look for the first result containing 'legistar.com'
                    for item in search_data['items']:
                        if 'link' in item and 'primegov.com' in item['link']:
                            parsed_url = urlparse(item['link'])
                            legistar_url = parsed_url.netloc  # Extract the domain part
                            break
            
            # Small delay to avoid rate limiting
            time.sleep(0.01)
        except Exception as e:
            print(f"Error searching for {city}: {e}")
        
        legistar_url_validated = np.nan
        # If a candidate URL was found, use OpenAI to check if it belongs to the city
        if not pd.isna(legistar_url):
            prompt = (f"Does the URL '{legistar_url}' seem to belong to the city '{city}'? of California"
                      "Answer with only 'yes' or 'no'."
                      "Example 1:"
                      "Prompt: Does the URL ci-ssf-ca.legistar.com seem to belong to the city 'City of South San Francisco' of California"
                      "Answer: yes\n"
                    "Example 2:"
                      "Prompt: Does the URL napa.ca.legistar.com seem to belong to the city 'Napa' of California"
                      "Answer: yes\n"
                     "Example 3:"
                     "Prompt: Does the URL san-marcos-tx.legistar.com seem to belong to the city 'San Marcos' of California"
                     "Answer: no\n")
            try:
                completion = openai.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=[
                        {"role": "system", "content": "You are a helpful assistant. Answer only 'yes' or 'no'."},
                        {"role": "user", "content": prompt}
                    ],
                    max_tokens=5,
                    temperature=0
                )
                answer = completion.choices[0].message.content.strip().lower()
                # If the answer is not 'yes', treat the URL as unrelated
                if answer != "yes":
                    legistar_url_validated = np.nan
                else:
                    legistar_url_validated = legistar_url
            except Exception as e:
                print(f"Error with OpenAI API for {city}: {e}")
        
        results.append({"city": city, 
                        "legistar_url": legistar_url, 
                        "legistar_url_validated": legistar_url_validated})
    
    return pd.DataFrame(results)


In [4]:
API_KEY = os.environ.get('GOOGLE_SEARCH_API_KEY')
cx = os.environ.get('GOOGLE_CUSTOM_SEARCH_ENGINE_NAME')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [ ]:
df = find_legistar_urls(cities, google_api_key=API_KEY, cx=cx, openai_token=OPENAI_API_KEY)

In [124]:
df.legistar_url_validated.notna().sum()

53

In [125]:
client_names = df['legistar_url'].str.split('.').str[0]

In [126]:
df.legistar_url.notna().sum(), df.legistar_url_validated.notna().sum()

(389, 53)

In [129]:
df.loc[df.legistar_url_validated.notna()]

,city,legistar_url,legistar_url_validated
2,Alameda,alameda.legistar.com,alameda.legistar.com
52,Burlingame,burlingameca.legistar.com,burlingameca.legistar.com
66,Carson,carson.legistar.com,carson.legistar.com
74,Chula Vista,chulavista.legistar.com,chulavista.legistar.com
76,Claremont,claremontca.legistar.com,claremontca.legistar.com
87,Commerce,cityofcommerce.legistar.com,cityofcommerce.legistar.com
95,Costa Mesa,costamesa.legistar.com,costamesa.legistar.com
100,Culver City,culver-city.legistar.com,culver-city.legistar.com
101,Cupertino,cupertino.legistar.com,cupertino.legistar.com
128,Emeryville,emeryville.legistar.com,emeryville.legistar.com


It's a hard case when two cities in different states share the same name. See the San Marcos Texas sneaking in.

Below was my first attempt (without GPT validating the answer). This is what currently populates the db's raw_event table but it's both too short of a list (it only covered the first 100 cities in the list), and too long of a list (because half of them are false matches that are now being filtered out by chatgpt)

In [77]:
first_hundred = ['hesperia', 'santabarbara', 'alameda', 'oakland', 'sanbernardino',
       'octa', np.nan, 'hayward', 'napa', 'cityoforange', 'sanbernardino',
       'wauwatosacitywi', 'contra-costa', 'sanbernardino',
       'sanbernardino', 'humboldt', 'ci-ssf-ca', 'redondo', 'tol',
       'sonoma-county', 'manteca-ca', 'cityofmerced', 'monterey',
       'carson', 'oakland', 'sbcera', 'solano', 'riversideca',
       'sunnyvaleca', 'sanbernardino', 'temeculaca', 'monterey',
       'longbeach', 'cityofmerced', 'longbeach', 'sfgov', 'solano',
       'actransit', 'metro', 'sanbernardino', 'sanbernardino',
       'cook-county', 'humboldt', 'fresno', 'longbeach', 'fresno',
       'cityoforange', 'contra-costa', 'sanmateocounty', 'santabarbara',
       'octa', 'sjrs', 'burlingameca', 'culver-city', np.nan, 'santaclara',
       np.nan, np.nan, 'santa-rosa', 'sanmateocounty', 'monterey',
       'riversideca', 'scvwd', 'sdcounty', 'monterey', 'santabarbara',
       'carson', 'culver-city', 'manteca-ca', 'metro', 'cityofmerced',
       'chino', 'chino', 'cityofmerced', 'chulavista', 'fontana',
       'claremontca', 'hayward', 'countyoflake', 'mendocino', 'fresno',
       'monterey', 'fresnocounty', 'nevco', 'ci-ssf-ca', 'sanbernardino',
       'sonoma-county', 'cityofcommerce', 'huntingtonbeach',
       'contra-costa', 'visalia', 'tehamacounty', 'corona',
       'portofsandiego', 'sfgov', 'costamesa', 'sonoma-county',
       'longbeach', 'oakland', 'metro', 'culver-city', 'cupertino',
       'actransit', 'sanmateocounty', 'sjrs', 'contra-costa']

In [28]:

def test_sunnyvale_legistar(api_key, cx):
    """
    Unit test to verify that the function returns 'sunnyvaleca.legistar.com'
    for the city of 'Sunnyvale'.
    
    Args:
        api_key (str): Google Custom Search API key
        cx (str): Google Custom Search engine ID
    """
    results = find_legistar_urls(['Sunnyvale'], api_key, cx)
    assert results.loc[0, 'legistar_url'] == 'sunnyvaleca.legistar.com', \
        f"Expected 'sunnyvaleca.legistar.com', got {results.loc[0, 'legistar_url']}"
    print("Test passed!")


In [29]:
test_sunnyvale_legistar(api_key=API_KEY, cx=cx)

Test passed!


In [7]:
results = find_legistar_urls(['Albany'], API_KEY, cx, OPENAI_API_KEY)
results

,city,legistar_url,legistar_url_validated
0,Albany,NaN,NaN
